In [1]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from collections import *
import math
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

# Get Data from Google Sheets

In [2]:
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

# Reference: https://developers.google.com/sheets/api/quickstart/python
def read_google_sheets(SPREADSHEET_ID, RANGE_NAME, HEADER_RANGE):
    creds = None
    # autogenerated
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('sheets', 'v4', credentials=creds)

    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                                range=RANGE_NAME).execute()

    header = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                               range=HEADER_RANGE).execute()

    header_values = header.get('values', [])
    values = result.get('values', [])

    return values, header_values

In [5]:
SPREADSHEET_ID = '1AjynK9mMQTw58B_B8b_ZIip3fyUm-aoV7Pp21HziBb0'
RANGE_NAME = 'canto_codings!A2:AT'
HEADER_RANGE = 'canto_codings!A1:AT1'

data, header = read_google_sheets(SPREADSHEET_ID, RANGE_NAME, HEADER_RANGE)

canto_data = pd.DataFrame(data, columns = header[0])

In [95]:
features = ['cv_'+str(i+1)for i in range(37)]
cultures = canto_data['Culture'].unique()

In [94]:
final_df = pd.DataFrame()
for culture in cultures:
    modal_profile = dict(canto_data[canto_data['Culture']==culture][features].mode().loc[0])
    modal_profile['culture'] = culture
    final_df = final_df.append(modal_profile, ignore_index=True)

In [99]:
final_df.to_csv('././output/modal_profiles.csv')